In [1]:
%%html
<h2>TensorFlow による実装 ：手書きの数字の画像データ判定<br>
活性化関数：ReLU<br>
隠れ層：3層<br>
dropout : ドロップアウトしない確率　学習時：0.5 テスト時：1.0
    学習時とテスト時でドロップアウト率が変動するので、placeholder で定義しておく。
</h2>

In [2]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(0)
tf.set_random_seed(1234)

'''
データの生成
'''
mnist = datasets.fetch_mldata('MNIST original', data_home='.')

n = len(mnist.data)
N = 10000  # MNISTの一部を使う
train_size = 0.8
indices = np.random.permutation(range(n))[:N]  # ランダムにN枚を選択

X = mnist.data[indices]
y = mnist.target[indices]
Y = np.eye(10)[y.astype(int)]  # 1-of-K 表現に変換

X_train, X_test, Y_train, Y_test =\
    train_test_split(X, Y, train_size=train_size)

'''
モデル設定
'''
n_in = len(X[0])  # 784
n_hidden = 200
n_out = len(Y[0])  # 10

"""
tf.nn.dropout(x, keep_prob, noise_shape=None, seed=None, name=None)
ドロップアウトを行う関数
 第1引数: x
  プーリング層からの出力をそのまま与えれば良い
 第2引数: keep_prob
  ロップアウトしない率
  
使用例
# 2割をドロップアウトする
tf.nn.dropout(res,0.8)
"""

x = tf.placeholder(tf.float32, shape=[None, n_in])
t = tf.placeholder(tf.float32, shape=[None, n_out])
keep_prob = tf.placeholder(tf.float32)  # ドロップアウトしない確率

# 入力層 - 隠れ層
W0 = tf.Variable(tf.truncated_normal([n_in, n_hidden], stddev=0.01))
b0 = tf.Variable(tf.zeros([n_hidden]))
h0 = tf.nn.relu(tf.matmul(x, W0) + b0)
h0_drop = tf.nn.dropout(h0, keep_prob)

# 隠れ層 - 隠れ層
W1 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b1 = tf.Variable(tf.zeros([n_hidden]))
h1 = tf.nn.relu(tf.matmul(h0_drop, W1) + b1)
h1_drop = tf.nn.dropout(h1, keep_prob)

W2 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b2 = tf.Variable(tf.zeros([n_hidden]))
h2 = tf.nn.relu(tf.matmul(h1_drop, W2) + b2)
h2_drop = tf.nn.dropout(h2, keep_prob)

# 隠れ層 - 出力層
W3 = tf.Variable(tf.truncated_normal([n_hidden, n_out], stddev=0.01))
b3 = tf.Variable(tf.zeros([n_out]))
y = tf.nn.softmax(tf.matmul(h2_drop, W3) + b3)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y),
                               reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

'''
モデル学習
'''
epochs = 30
batch_size = 200

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

n_batches = (int)(N * train_size) // batch_size

for epoch in range(epochs):
    X_, Y_ = shuffle(X_train, Y_train)

    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size

        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end],
            keep_prob: 0.5
        })

    # 訓練データに対する学習の進み具合を出力
    loss = cross_entropy.eval(session=sess, feed_dict={
        x: X_,
        t: Y_,
        keep_prob: 1.0
    })
    acc = accuracy.eval(session=sess, feed_dict={
        x: X_,
        t: Y_,
        keep_prob: 1.0
    })
    print('epoch:', epoch, ' loss:', loss, ' accuracy:', acc)

'''
予測精度の評価
'''
accuracy_rate = accuracy.eval(session=sess, feed_dict={
    x: X_test,
    t: Y_test,
    keep_prob: 1.0
})
print('accuracy: ', accuracy_rate)

epoch: 0  loss: 2.28336  accuracy: 0.28275
epoch: 1  loss: 2.16672  accuracy: 0.4175
epoch: 2  loss: 1.17982  accuracy: 0.625125
epoch: 3  loss: 0.768325  accuracy: 0.76575
epoch: 4  loss: 0.571193  accuracy: 0.834
epoch: 5  loss: 0.47362  accuracy: 0.86575
epoch: 6  loss: 0.422134  accuracy: 0.871
epoch: 7  loss: 0.346748  accuracy: 0.899625
epoch: 8  loss: 0.309143  accuracy: 0.91075
epoch: 9  loss: 0.283576  accuracy: 0.9185
epoch: 10  loss: 0.262914  accuracy: 0.926625
epoch: 11  loss: 0.237035  accuracy: 0.933125
epoch: 12  loss: 0.22274  accuracy: 0.934125
epoch: 13  loss: 0.207128  accuracy: 0.941375
epoch: 14  loss: 0.199309  accuracy: 0.94275
epoch: 15  loss: 0.177689  accuracy: 0.948125
epoch: 16  loss: 0.162945  accuracy: 0.95525
epoch: 17  loss: 0.153302  accuracy: 0.955875
epoch: 18  loss: 0.142194  accuracy: 0.958875
epoch: 19  loss: 0.129509  accuracy: 0.963125
epoch: 20  loss: 0.127136  accuracy: 0.964
epoch: 21  loss: 0.116768  accuracy: 0.96625
epoch: 22  loss: 0.1067